In [103]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [104]:
df = pd.read_csv('outputs/dataset_20210523.csv')
df

,game,blueTop,blueTopGP,blueTopWR,blueTopKDA,blueJungle,blueJungleGP,blueJungleWR,blueJungleKDA,blueMid,...,redMidKDA,redAdc,redAdcGP,redAdcWR,redAdcKDA,redSupport,redSupportGP,redSupportWR,redSupportKDA,result
0,ESPORTSTMNT03/1632489,82,8,0.50,2.9,104,27,0.85,5.9,142,...,3.6,81,21,0.57,4.3,89,10,0.60,2.5,0
1,ESPORTSTMNT03/1632500,79,14,0.36,2.6,104,63,0.63,5.1,68,...,4.9,21,10,0.70,5.3,3,11,0.55,2.8,1
2,ESPORTSTMNT03/1632502,79,14,0.36,2.6,104,63,0.63,5.1,142,...,4.4,21,10,0.70,5.3,3,11,0.55,2.8,1
3,6909-9183,43,2,0.00,0.9,80,5,0.60,3.9,134,...,5.2,145,43,0.60,6.3,3,4,0.75,4.5,1
4,6909-9184,24,16,0.56,2.8,120,11,0.55,3.6,61,...,5.2,145,43,0.60,6.3,57,2,0.50,1.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,7350-9936,58,31,0.32,2.4,77,12,0.50,4.1,134,...,4.1,18,4,0.25,3.3,12,6,0.50,1.9,0
4344,ESPORTSTMNT01/1887606,14,4,1.00,7.9,120,6,0.50,2.8,112,...,2.6,18,0,0.00,0.0,875,1,0.00,2.5,0
4345,ESPORTSTMNT05/1860602,39,1,0.00,0.4,141,3,0.00,1.5,134,...,3.4,119,1,1.00,7.0,89,1,1.00,15.0,1
4346,ESPORTSTMNT05/1860625,58,3,0.33,3.1,876,3,0.33,3.6,61,...,0.3,145,3,0.00,1.1,12,3,0.00,1.2,0


In [105]:
y = df['result'].copy()
X = df.drop(['blueTop', 'blueJungle', 'blueMid', 'blueADC', 'blueSupport', 'redTop', 'redJungle', 'redMid', 'redAdc', 'redSupport', 'result', 'game'],axis=1)
X

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,8,0.50,2.9,27,0.85,5.9,59,0.76,10.7,10,...,4.3,13,0.46,3.6,21,0.57,4.3,10,0.60,2.5
1,14,0.36,2.6,63,0.63,5.1,1,0.00,1.0,20,...,4.1,55,0.69,4.9,10,0.70,5.3,11,0.55,2.8
2,14,0.36,2.6,63,0.63,5.1,5,0.40,2.9,2,...,6.4,3,0.67,4.4,10,0.70,5.3,11,0.55,2.8
3,2,0.00,0.9,5,0.60,3.9,36,0.75,6.2,5,...,3.0,6,0.67,5.2,43,0.60,6.3,4,0.75,4.5
4,16,0.56,2.8,11,0.55,3.6,50,0.64,6.1,5,...,2.9,6,0.67,5.2,43,0.60,6.3,2,0.50,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,31,0.32,2.4,12,0.50,4.1,4,0.75,4.0,0,...,5.4,3,0.67,4.1,4,0.25,3.3,6,0.50,1.9
4344,4,1.00,7.9,6,0.50,2.8,4,0.50,3.6,7,...,3.8,15,0.33,2.6,0,0.00,0.0,1,0.00,2.5
4345,1,0.00,0.4,3,0.00,1.5,5,0.20,3.2,2,...,1.9,3,0.67,3.4,1,1.00,7.0,1,1.00,15.0
4346,3,0.33,3.1,3,0.33,3.6,5,0.40,2.7,2,...,0.1,1,0.00,0.3,3,0.00,1.1,3,0.00,1.2


In [106]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      blueTopGP  blueTopWR  blueTopKDA  blueJungleGP  blueJungleWR  \
1115          1       1.00         7.0             3          0.67   
4255          8       0.38         2.0             5          0.60   
4213          2       0.50         2.4            14          0.79   
2658          5       0.20         3.0            10          0.60   
432           1       0.00         0.3             6          0.50   
...         ...        ...         ...           ...           ...   
79            5       0.40         1.5            10          0.80   
1957          5       0.60         4.8             6          0.33   
2115         23       0.74         4.1            22          0.68   
970           7       0.57         5.1             2          1.00   
490           2       0.00         0.8             3          0.00   

      blueJungleKDA  blueMidGP  blueMidWR  blueMidKDA  blueADCGP  ...  \
1115            3.3          3       1.00         6.1          3  ...   
4255         

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
C:\Users\linco\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [108]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [111]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
scaler = StandardScaler()    
scaler.fit(X_train)

# DIA 1 (05/06)
# FLAMENGO VS PAIN
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu errei, a distribuição de probabilidades para FLAMENGO VS PAIN era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# KABUM VS VORAX
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu errei, a distribuição de probabilidades para KABUM VS VORAX era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# MINERS VS FURIA
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu acertei, a distribuição de probabilidades para MINERS VS FURIA era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# RENSGA VS INTZ
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu acertei, a distribuição de probabilidades para RENSGA VS INTZ era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# LOUD VS RED
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu acertei, a distribuição de probabilidades para LOUD VS RED era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# DIA 2 (06/06)
# PAIN VS VORAX
match_to_predict = {'blueTopGP': [2],'blueTopWR': [0],'blueTopKDA': [0.9],'blueJungleGP': [0],'blueJungleWR': [0],'blueJungleKDA': [0],'blueMidGP': [6],'blueMidWR': [0.17],'blueMidKDA': [3.2],'blueADCGP': [5],'blueADCWR': [0.4],'blueADCKDA': [1.9],'blueSupportGP': [0],'blueSupportWR': [0],'blueSupportKDA': [0],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu acertei, a distribuição de probabilidades para RENSGA VS MINERS era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# RENSGA VS MINERS
match_to_predict = {'blueTopGP': [],'blueTopWR': [],'blueTopKDA': [],'blueJungleGP': [],'blueJungleWR': [],'blueJungleKDA': [],'blueMidGP': [],'blueMidWR': [],'blueMidKDA': [],'blueADCGP': [],'blueADCWR': [],'blueADCKDA': [],'blueSupportGP': [],'blueSupportWR': [],'blueSupportKDA': [],'redTopGP': [],'redTopWR': [],'redTopKDA': [],'redJungleGP': [],'redJungleWR': [],'redJungleKDA': [],'redMidGP': [],'redMidWR': [],'redMidKDA': [],'redAdcGP': [],'redAdcWR': [],'redAdcKDA': [],'redSupportGP': [],'redSupportWR': [],'redSupportKDA': []}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('Eu acertei, a distribuição de probabilidades para RENSGA VS MINERS era:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

Eu errei, a distribuição de probabilidades para FLAMENGO VS PAIN era:
[[0.56353744 0.43646256]]
[0]
Eu errei, a distribuição de probabilidades para KABUM VS VORAX era:
[[0.16119663 0.83880337]]
[1]
Eu acertei, a distribuição de probabilidades para MINERS VS FURIA era:
[[0.61289374 0.38710626]]
[0]
Eu acertei, a distribuição de probabilidades para RENSGA VS INTZ era:
[[0.20498652 0.79501348]]
[1]
Eu acertei, a distribuição de probabilidades para LOUD VS RED era:
[[0.47086256 0.52913744]]
[1]


In [110]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.926437,0.921182,0.921182,0.921182
1,Support Vector Machine (Linear Kernel),0.927586,0.921376,0.923645,0.922509
2,Support Vector Machine (RBF Kernel),0.924138,0.912621,0.926108,0.919315
3,Decission Tree,0.877011,0.863747,0.874384,0.869033
4,Adaboost,0.931034,0.932500,0.918719,0.925558
5,Random Forest,0.934483,0.950904,0.906404,0.928121
6,Gradient Boosting Classifier,0.945977,0.943210,0.940887,0.942047
